# Class Activation Map (CAM)

This is one of many ways to visualize and get insights from a Convolutional Neural Network. What this basically does is that it creates a heatmap of "Class Activation" over the input image.  A "class activation" heatmap is a 2D grid of scores associated with an specific output class, computed for every location in any input image, indicating how important each location is with respect to the class considered. Quite simply, it tells us which features the model is looking for. Keras will be the deep framing framework that is going to be utilized. I am starting to really love Keras, its just so easy to use and as an effect saves a lot of time coding. If tensorflow was used for this project it would have taken longer because tensorflow is not rich in helper functions. Everything that is needed for this are all in Keras.

There are a lot of interesting insights from the model that are shown towards the end of this exercise.

In [1]:
from keras.applications.vgg16 import VGG16
import matplotlib.image as mpimg
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
K.clear_session()

### For this exercise we will be using the VGG16 model

In [2]:
model = VGG16(weights='imagenet')

553467904/553467096 [==============================] - 5s 0us/step


### Sample Image

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
img_path = 'G:/hummingbird.jpg'
img=mpimg.imread(img_path)
plt.imshow(img)

FileNotFoundError: ignored

### Resizing image to fit the input size of VGG

In [ ]:
from keras.preprocessing import image
img = image.load_img(img_path, target_size=(224, 224))

### Convert to numpy array

In [ ]:
x = image.img_to_array(img)

### Reshape data so that it is in "batch" form because the model only accepts input in this form

In [ ]:
import numpy as np
x = np.expand_dims(x, axis=0)

### "Batch" form

In [ ]:
x.shape

### Preprocessing

In [ ]:
from keras.applications.vgg16 import preprocess_input
x = preprocess_input(x)

In [ ]:
x

### Prediction

In [ ]:
import pandas as pd
from keras.applications.vgg16 import decode_predictions
preds = model.predict(x)
predictions = pd.DataFrame(decode_predictions(preds, top=3)[0],columns=['col1','category','probability']).iloc[:,1:]
print('PREDICTION:',predictions.loc[0,'category'])

In [ ]:
import seaborn as sns
f = sns.barplot(x='probability',y='category',data=predictions,color="red")
sns.set_style(style='white')
f.grid(False)
f.spines["top"].set_visible(False)
f.spines["right"].set_visible(False)
f.spines["bottom"].set_visible(False)
f.spines["left"].set_visible(False)
f.set_title('Top 3 Predictions:')

### Index of the prediction

In [ ]:
argmax = np.argmax(preds[0])

### Get the index of the prediction

In [ ]:
output = model.output[:, argmax]

### Model Archtecture

In [ ]:
model.layers_by_depth

### We want to get the final Convolutional Layer

In [ ]:
last_conv_layer = model.get_layer('block5_conv3')

### Get the Gradient

In [ ]:
grads = K.gradients(output, last_conv_layer.output)[0]

### Each entry of this tensor is the mean intensity of the gradient over a specific feature map channel. This has a shape of (512,)

In [ ]:
pooled_grads = K.mean(grads, axis=(0, 1, 2))

### Access the values of the quantities we just defined

In [ ]:
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

### These are the values of these two quantities, as Numpy arrays, given our sample image of two elephants

In [ ]:
pooled_grads_value, conv_layer_output_value = iterate([x])

### We multiply each channel in the feature map array by "how important this channel is" with regard to the elephant class

In [ ]:
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

### Plotting the Heatmap

In [ ]:
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

### Load Image with CV2

In [ ]:
import cv2
img = cv2.imread(img_path)

### Resize the heatmap

In [ ]:
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

### Convert heatmap to RGB

In [ ]:
heatmap = np.uint8(255 * heatmap)

### Apply heatmap to original Image

In [ ]:
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

### Apply heatmap intensity factor

In [ ]:
hif = .8

In [ ]:
superimposed_img = heatmap * hif + img

### Save to disk

In [ ]:
output = 'G:/output.jpeg'
cv2.imwrite(output, superimposed_img)

img=mpimg.imread(output)

### Plot

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.title(predictions.loc[0,'category'])

### Putting it all in a single funtion

In [ ]:
def cam(img_path):
    from keras.applications.vgg16 import VGG16
    import matplotlib.image as mpimg
    from keras import backend as K
    import matplotlib.pyplot as plt
    %matplotlib inline
    K.clear_session()
    
    model = VGG16(weights='imagenet')
    img=mpimg.imread(img_path)
    plt.imshow(img)
    from keras.preprocessing import image
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    from keras.applications.vgg16 import preprocess_input
    x = preprocess_input(x)
    preds = model.predict(x)
    predictions = pd.DataFrame(decode_predictions(preds, top=3)[0],columns=['col1','category','probability']).iloc[:,1:]
    argmax = np.argmax(preds[0])
    output = model.output[:, argmax]
    last_conv_layer = model.get_layer('block5_conv3')
    grads = K.gradients(output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    for i in range(512):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    import cv2
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    hif = .8
    superimposed_img = heatmap * hif + img
    output = 'G:/output.jpeg'
    cv2.imwrite(output, superimposed_img)
    img=mpimg.imread(output)
    plt.imshow(img)
    plt.axis('off')
    plt.title(predictions.loc[0,'category'].upper())
    return None

### Trying it out on other images

In [ ]:
cam('G:/sample1.jpg')

In [ ]:
cam('G:/sample2.jpg')

In [ ]:
cam('G:/sample3.jpg')

### Using it on the same Category

As we expected, the CNN is looking for specific features. In the example below, when it sees this kind of nose/mouth area of a dog the models predicts that it is a greater swiss mountain dog.

In [ ]:
cam('G:/greater_swiss_mountain_dog.jpg')

In [ ]:
cam('G:/greater_swiss_mountain_dog4.jpg')

In [ ]:
cam('G:/greater_swiss_mountain_dog2.jpg')

In [ ]:
cam('G:/greater_swiss_mountain_dog3.jpg')

### Dogs

It is interesting what the model is looking for in order to identfy the breed of the dog.

In [ ]:
cam('G:/african_hunting_dog.jpg')

In [ ]:
cam('G:/bernese_mountain_dog.jpg')

In [ ]:
cam('G:/french_bulldog2.jpg')

In [ ]:
cam('G:/Maltese_puppy.jpeg')